Math implementation 

Reference from Ru Paper that proposes the novel CoCabo Approach. Tehre are further literature suggestions:

Carlucci et al. 2020 further explaining MAB philosophy and how categorical values are coordinated inthe multi agent system

Auer et al. 2002b where EXP3 originates from one of the original ideas in where a a categorical variable is selected by MAB constructing a GP surrogate specific to the chosen cateogry for modelling the continuous domain without sharing information across different categories (however this incurs quite a bit of computation)

Nguyen et al 2019 proposes a Thmpson sampling method to BO using the above approach

They use a general adversarial bandit setting (assumption from EXP3) to handle the non-stationary-ness of the rewards in categorical variables (Allesiardo et al 2017 for more info on this).

Critically this allows the MAB to make fewere assumptions on reward distributions, allowing for cheaper computation. But most importantly, the MAB decides values for categorical inpputs, therefore we only need to optimize the acquisition function over the continuous subspace (big X, maybe it's a CHI), which is a subspace of all Real data (given our limit of requiring experimental data to fill information) this allows for a mathematically sound approach to efficiently compute over categorical and continuous values

Let's read Appendix C and implement some of its math to make sense of it.

Below is the categorical kernel, trying to work out how to write the similarity metric (=category_overlap_kernel) if categories are the same the matrix shows this as 1s and 0s (if they are different categories) (This is derived as a special case of an RBF kernel)

In [ ]:
category_overlap_matrix = X_cat[:, None] == X_cat[None, :] # I believe this is the category overlap matrix (while below is the full kernel)

k_cat = (self.overlap_variance / X_cat.shape[1]) * np.sum(X_cat[:, None] == X_cat[None, :], axis=-1)  # Normalizes overlap Formula from Ru supplementary


This one i extracted from my previous work so variables names are messed up and won't work here.

The below should work as proposed in Ru 2020, formula 2

In [ ]:
# Combine the Matern and Category Overlap Kernels using the formula
K = (1 - self.lambda_param) * (matern_matrix + category_overlap_matrix) + (self.lambda_param * matern_matrix * category_overlap_matrix)  # Ru's multiplication + summation


The batch Cocabo is implemented using 'multiplays' formulation of EXP3 called EXP3.M (Auer et al 2002) So let's study there formulation below.

This is said to return a batch of categorical chocies and combines it with the Kriging Believer (Ginsbourger et al 2010) a batch method to select batch points in the continuous domain.

Ru et al 2020 chose this formulation for batch creation as it can consider already-selected batch points, including those with different categorical values without making restrictions (most popular methods assumes that f is Lipschitz continuous)